In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go


In [2]:
df = pd.read_csv("./barcelona/bus_stops.csv")
df

,Code,Transport,Longitude,Latitude,Bus.Stop,District.Name,Neighborhood.Name
0,K014,Day bus stop,2.171619,41.413744,BUS -192--,Horta-Guinardó,el Guinardó
1,K014,Day bus stop,2.134902,41.420222,BUS -124--,Gràcia,Vallcarca i els Penitents
2,K014,Day bus stop,2.162913,41.423187,BUS -117--,Horta-Guinardó,la Font d'en Fargues
3,K014,Day bus stop,2.163667,41.422899,BUS -117--,Horta-Guinardó,la Font d'en Fargues
4,K014,Day bus stop,2.120212,41.397209,BUS -130--,Sarrià-Sant Gervasi,Sarrià
...,...,...,...,...,...,...,...
3157,K014,Day bus stop,2.183887,41.422024,BUS -191--,Sant Andreu,el Congrés i els Indians
3158,K014,Day bus stop,2.176710,41.446419,BUS -127-81--,Nou Barris,les Roquetes
3159,K015,Night bus stop,2.144596,41.411446,NITBUS -N4--,Gràcia,Vallcarca i els Penitents
3160,K014,Day bus stop,2.142256,41.371176,BUS -91-115-V5--,Sants-Montjuïc,la Bordeta


In [13]:

fig = go.Figure(data=go.Scattergeo(
        lon = df['Longitude'],
        lat = df['Latitude'],
        text = df['Bus.Stop'],
        marker = dict(size = 3), 
        locationmode = 'country names'
        
        ))

fig.update_layout(
        geo_scope='europe',
        autosize = True
        
    )
fig.show()

In [22]:
df_life = pd.read_csv("./barcelona/life_expectancy.csv")
df_accidents = pd.read_csv("./barcelona/accidents_2017.csv")

districts = list(set(df_accidents['District Name'].values))

neighborhoods = list(set(df_life['Neighborhood'].values))


# for iets in neighborhood:
    

In [59]:
df_accidents = df_accidents[df_accidents["Neighborhood Name"] != "Unknown"]

counts = df_accidents.groupby(['District Name']).count()
counts
df_counts = counts['Id']
print(df_counts)

new_index = ['Ciutat Vella', 'Sant Martí', 'Sants-Montjuïc', 'Eixample', 'Sant Andreu', 'Les Corts', 'Gràcia', 'Horta-Guinardó', 'Sarrià-Sant Gervasi', 'Nou Barris'  ]
df_counts = df_counts.reindex(new_index)
print(df_counts)

fig = go.Figure([go.Bar(x=df_counts.index, y = df_counts.values)])
fig.show()

District Name
Ciutat Vella            600
Eixample               3029
Gràcia                  531
Horta-Guinardó          743
Les Corts               726
Nou Barris              542
Sant Andreu             597
Sant Martí             1336
Sants-Montjuïc         1104
Sarrià-Sant Gervasi    1104
Name: Id, dtype: int64
District Name
Ciutat Vella            600
Sant Martí             1336
Sants-Montjuïc         1104
Eixample               3029
Sant Andreu             597
Les Corts               726
Gràcia                  531
Horta-Guinardó          743
Sarrià-Sant Gervasi    1104
Nou Barris              542
Name: Id, dtype: int64


In [ ]:
#lijn maken in bovenstaande grafiek